In [2]:
# Progetto Data Science Lab
## Sentiment Analysis Corona Virus
## Giacomo De Cobbi
## Alessandro Pontini
## Kevin Tranchina

In [3]:
import os 
import pandas as pd
from datetime import datetime, timedelta
import nltk
import re ##regulare expression
import string
from nltk.tokenize import BlanklineTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import subprocess
import operator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
subprocess.Popen(r'explorer /select,"C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic"')

# Import and Preprocessing Dataset

In [126]:
# imposta il path dove vuoi hai i dati

path = str(r'C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic')
path = path.replace("\\","/")
os.chdir(path)

In [127]:
# Lettura file 
# 33716 righe
vaccination = pd.read_csv("vaccination_all_tweets.csv")

In [128]:
vaccination.loc[0,'text']

'Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF'

## Preprocessiong
### Rimozione Hash, rimozione di tutto i tag e i link
#### Esempio con 
'Ciao mi chiamo @Antionio Antonio #Ciao #Home @Nicole https://facebook.com'

In [129]:
esempio = 'Ciao mi chiamo @Giuseppe Antonio #Ciao #Home @Nicole https://facebook.com'
print(esempio)
# @[A-Za-z0-9]+ rimuove tutte le parole(caratteri alfanumerici) che cominciano con @
# #[A-Za-z0-9]+ rimuove tutte le parole(caratteri alfanumerici) che cominciano con #
# [^0-9A-Za-z ] tiene solo i caratteri alfanumerici
# (\w+:\/\/\S+) rimozione link

esempio = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z ])|(\w+:\/\/\S+)","",esempio)

# fa la stessa cosa di sopra, ma più pulito
# esempio = re.sub("(@\w+)| (#\w+)|([^\w ])|(\w+:\/\/\S+)","",esempio)

print(esempio)


# Rimuove anche gli hashtag interi
# esempio = re.sub("(@[A-Za-z0-9]+)| (#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",esempio)

# Rimozione degli spazi in più
esempio = re.sub(' +',' ',esempio)
print(esempio)

Ciao mi chiamo @Giuseppe Antonio #Ciao #Home @Nicole https://facebook.com
Ciao mi chiamo  Antonio Ciao Home  
Ciao mi chiamo Antonio Ciao Home 


#### Applicazione al nosto dataset

In [130]:
# rimozione del cancelletto nell'hashtag 
# rimozione dei link e dei tag

reNoHashNoTagNoLink = "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"
reNo_HashTAG_NoTagNoLink = "(@[A-Za-z0-9]+)| (#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"
vaccination.loc[:,'text'] = vaccination.loc[:,'text'].apply(lambda x : ''.join(re.sub(reNo_HashTAG_NoTagNoLink,"",x)))
stampa = vaccination.loc[0:10,'text'].apply(lambda x : print(x))

Same folks said daikon paste could treat a cytokine storm 
While the world has been on the wrong side of history this year hopefully the biggest vaccination effort weve ev 
coronavirus19 Russian vaccine is created to last 24 years 
Facts are immutable Senator even when youre not ethically sturdy enough to acknowledge them 1 You were born i 
Explain to me again why we need a vaccine   
Does anyone have any useful adviceguidance for whether the COVID vaccine is safe whilst breastfeeding 
it is a bit sad to claim the fame for success of on patriotic competition between USA Canada UK and 
There have not been many bright days in 2020 but here are some of the best 1 winning 
Covid vaccine You getting it
CovidVaccine States will start getting Monday says pakustv 
while deaths are closing in on the 300000 mark millions of people wait The first U 


In [131]:
## lowerCase
# vaccination.text = vaccination.text.str.lower()

## rimozione dei numeri
vaccination.text = vaccination.text.replace(r'\d+','')

## rimozione spazi bianchi
vaccination.text  = vaccination.text.astype(str).str.strip()

## rimozione doppi spazi bianchi
vaccination.text = vaccination.text.replace(' +',' ', regex=True)

# rimozione punteggiatura
# verificato rimuove anche i cancelletti
vaccination.text=vaccination.text.astype(str).str.translate(str.maketrans('','',string.punctuation))

#rimozione \n
vaccination.text = vaccination.text.replace("\n"," ")

#rimozione apice strano
vaccination.text = vaccination.text.replace("'’'"," ")

0        Same folks said daikon paste could treat a cyt...
1        While the world has been on the wrong side of ...
2        coronavirus19 Russian vaccine is created to la...
3        Facts are immutable Senator even when youre no...
4                Explain to me again why we need a vaccine
                               ...                        
33712    BREAKING The information attack targeting will...
33713    Pity as my personal preference would have been...
33714           WHO in close contact with on certification
33715    URGENT US targets Europe in smear campaign aga...
33716    BREAKING US Targets Europe in Smear Campaign A...
Name: text, Length: 33717, dtype: object

In [11]:
# TweetTokenizer
tknzr = TweetTokenizer()
vaccination['tweet_tokenize'] = vaccination['text'].apply(tknzr.tokenize)
vaccination['tweet_tokenize']

0        [Same, folks, said, daikon, paste, could, trea...
1        [While, the, world, has, been, on, the, wrong,...
2        [coronavirus, SputnikV, AstraZeneca, PfizerBio...
3        [Facts, are, immutable, Senator, even, when, y...
4        [Explain, to, me, again, why, we, need, a, vac...
                               ...                        
33712    [BREAKING, The, information, attack, targeting...
33713    [Pity, as, my, personal, preference, would, ha...
33714    [WHO, in, close, contact, with, Russia, on, Sp...
33715    [URGENT, US, targets, Europe, in, smear, campa...
33716    [BREAKING, US, Targets, Europe, in, Smear, Cam...
Name: tweet_tokenize, Length: 33717, dtype: object

In [12]:
# Rimozione Stop Words
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words += ['im','youre','thats','get','cant','dont','go',
               'going','gonna','like','got', 'come', 'look','see','theres',
               'say','want','wanna','make','yes','right','']


vaccination['tweet_without_stopwords'] = vaccination['tweet_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
vaccination['tweet_without_stopwords'] 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [Same, folks, said, daikon, paste, could, trea...
1        [While, world, wrong, side, history, year, hop...
2        [coronavirus, SputnikV, AstraZeneca, PfizerBio...
3        [Facts, immutable, Senator, even, ethically, s...
4        [Explain, need, vaccine, whereareallthesickpeo...
                               ...                        
33712    [BREAKING, The, information, attack, targeting...
33713    [Pity, personal, preference, would, AstraZenec...
33714    [WHO, close, contact, Russia, SputnikV, certif...
33715    [URGENT, US, targets, Europe, smear, campaign,...
33716    [BREAKING, US, Targets, Europe, Smear, Campaig...
Name: tweet_without_stopwords, Length: 33717, dtype: object

# Sentiment Analysis
## VADER ( Valence Aware Dictionary for Sentiment Reasoning)
#### VADER  is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.

#### To calculate the sentimental score of the entire text, Vader scans the text for known sentimental features, modified the intensity and polarity according to the rules, summed up the scores of features found within the text and normalized the final score to (-1, 1) using function:



$$ compound=\frac{x}{\sqrt{x^2 +a}}$$

 

#### In Vader, alpha is set to be 15 which approximates the maximum expected value of x.

In [13]:
vaccination['scores']  = vaccination['text'] .apply(lambda review: sid.polarity_scores(review))

In [14]:
vaccination['scores']

0        {'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...
1        {'neg': 0.13, 'neu': 0.756, 'pos': 0.113, 'com...
2        {'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'comp...
3        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                               ...                        
33712    {'neg': 0.181, 'neu': 0.819, 'pos': 0.0, 'comp...
33713    {'neg': 0.096, 'neu': 0.904, 'pos': 0.0, 'comp...
33714    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
33715    {'neg': 0.147, 'neu': 0.705, 'pos': 0.149, 'co...
33716    {'neg': 0.161, 'neu': 0.839, 'pos': 0.0, 'comp...
Name: scores, Length: 33717, dtype: object

In [15]:
# Test
a = 'I love my girl'
print(sid.polarity_scores(a))

#-0,5<x<+0.5 

{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}


In [16]:
vaccination['compound']  = vaccination['scores'].apply(lambda score_dict: score_dict['compound'])
vaccination['compound']

0        0.4019
1       -0.1027
2        0.2500
3        0.0000
4        0.0000
          ...  
33712   -0.4767
33713   -0.1531
33714    0.0000
33715    0.0085
33716   -0.3612
Name: compound, Length: 33717, dtype: float64

#### Aggregazioni e analisi

In [17]:
print(vaccination.columns)
print(vaccination.info())
print('\nDescribe\n')
print(vaccination['user_location'].describe())

Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet', 'tweet_tokenize', 'tweet_without_stopwords', 'scores',
       'compound'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33717 entries, 0 to 33716
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       33717 non-null  int64  
 1   user_name                33717 non-null  object 
 2   user_location            25937 non-null  object 
 3   user_description         31402 non-null  object 
 4   user_created             33717 non-null  object 
 5   user_followers           33717 non-null  int64  
 6   user_friends             33717 non-null  int64  
 7   user_favourites          33717 non-null  int64  
 8   us

In [18]:
vaccination = vaccination[vaccination['user_location'].notna()]
#vaccination['user_location2'] = vaccination['user_location'].dropna()
vaccination.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25937 entries, 0 to 33716
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       25937 non-null  int64  
 1   user_name                25937 non-null  object 
 2   user_location            25937 non-null  object 
 3   user_description         24970 non-null  object 
 4   user_created             25937 non-null  object 
 5   user_followers           25937 non-null  int64  
 6   user_friends             25937 non-null  int64  
 7   user_favourites          25937 non-null  int64  
 8   user_verified            25937 non-null  bool   
 9   date                     25937 non-null  object 
 10  text                     25937 non-null  object 
 11  hashtags                 20371 non-null  object 
 12  source                   25929 non-null  object 
 13  retweets                 25937 non-null  int64  
 14  favorites             

In [19]:
vaccination.loc[vaccination['user_location'].str.contains('Italy', regex=True, na=False)]
vaccination.loc[0,'text']

'Same folks said daikon paste could treat a cytokine storm PfizerBioNTech'

In [20]:
vaccination['hashtags']

0                                       ['PfizerBioNTech']
1                                                      NaN
2        ['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...
3                                                      NaN
5                                                      NaN
                               ...                        
33708                              ['Zakharova', 'Russia']
33712                       ['BREAKING', 'SputnikV', 'US']
33713                 ['AstraZeneca', 'vaccine', 'Pfizer']
33714                        ['WHO', 'Russia', 'SputnikV']
33716             ['BREAKING', 'US', 'Russia', 'SputnikV']
Name: hashtags, Length: 25937, dtype: object